In [6]:
from glob import glob
from os import path
import seaborn as sns
import json
import pandas as pd
import os
import numpy as np
import re
from os.path import basename, splitext

# from solver import Instance

from argparse import Namespace
import sys
sys.path.append('../solver')
# from solver_output import practice_print
from solver_rostering import run_roster_solver_objval

In [7]:
list_city_dt = []

for file_in in os.listdir(r'../instances'):
    base_file = splitext(basename(file_in))[0]
    base_file_split = base_file.split('_')
    file = base_file_split[0]+'_'+base_file_split[1]
    list_city_dt.append(file)

list_city_dt = list(set(list_city_dt))
list_city_dt.sort()
print(list_city_dt)
print(len(list_city_dt))


['berlin_db=0.50', 'berlin_db=1.00', 'berlin_db=2.00', 'berlin_db=4.00', 'frankfurt_db=0.50', 'frankfurt_db=1.00', 'frankfurt_db=2.00', 'frankfurt_db=4.00', 'lyon_db=0.50', 'lyon_db=1.00', 'lyon_db=2.00', 'lyon_db=4.00', 'paris_db=0.50', 'paris_db=1.00', 'paris_db=2.00', 'paris_db=4.00']
16


In [8]:
#Subset to France

list_city_dt_subset = []

for city_db in list_city_dt:
    if (city_db.find('paris') > -1):
        list_city_dt_subset.append(city_db)

print(list_city_dt_subset)

['paris_db=0.50', 'paris_db=1.00', 'paris_db=2.00', 'paris_db=4.00']


In [9]:

def dict_append_workforce(dict_, base_file, city, demand_baseline, OC, RM, GM, model, max_n_shifts, workforce_size, objvalprev, objval, objvalnext):
    dict_['instance_file_base'].append(base_file)
    dict_['city'].append(city)
    dict_['demand_baseline'].append(demand_baseline)
    dict_['model'].append(model)
    dict_['max_n_shifts'].append(max_n_shifts)
    dict_['outsourcing_cost_multiplier'].append(OC)
    dict_['regional_multiplier'].append(RM)
    dict_['global_multiplier'].append(GM)
    dict_['workforce_size_region0'].append(workforce_size)
    dict_['objective_value_prev'].append(objvalprev)
    dict_['objective_value'].append(objval)
    dict_['objective_value_next'].append(objvalnext)
    return dict_

In [10]:
#Code that determines optimal workforce size

MAX_TRIES = 300

RM = 1.5
GM = 0.8
h_min = 32
h_max = 48
max_n_diff = 3

#for base_file in list_city_dt_subset:
for base_file in ['paris_db=4.00']:
    print(base_file+'============================================')

    if f'{base_file}.json' not in os.listdir(r'../workforce_size'):

        list_results = []

        #create inputs to run a trial workforce size
        weekday_ = f'{base_file}_dt=doublepeak.json'
        weekend_ = f'{base_file}_dt=uniform.json'

        instance_file_weekday = f"../instances/{weekday_}"
        instance_file_weekend = f"../instances/{weekend_}"

        shift_file_weekday = f"../shifts/{weekday_}"
        shift_file_weekend = f"../shifts/{weekend_}"

        #create dictionary for evaluating optimal workforce size
        workforce_out = f"../workforce_size/{base_file}.json"
        dict_out = {
            'instance_file_base':[],
            'city':[],
            'demand_baseline':[],
            'model':[],
            'max_n_shifts':[],
            'outsourcing_cost_multiplier':[],
            'regional_multiplier':[],
            'global_multiplier':[],
            'workforce_size_region0':[],
            'objective_value_prev':[],
            'objective_value':[],
            'objective_value_next':[]
        }

        city_pattern = r'(\w+)_db'
        db_pattern = r'db=(\d+\.\d+)'

        city_match = re.search(city_pattern, base_file)
        db_match = re.search(db_pattern, base_file)

        city = city_match.group(1) if city_match else None
        demand_baseline = float(db_match.group(1)) if db_match else None

        #Set min_tries based on demand_baseline
        if demand_baseline == 0.5:
            MIN_TRIES = 25
        elif demand_baseline == 1.0:
            MIN_TRIES = 50
        elif demand_baseline == 2.0:
            MIN_TRIES = 90
        elif demand_baseline == 4.0:
            MIN_TRIES = 195

        for OC in [1.2, 1.5, 1.8, 2.0, 2.5]:
        # for OC in [1.2]:
            print(f'OC = {OC}++++++++++++++++++++++')
            for model in ['fixed', 'flex', 'partflex']:

                if model == 'fixed':
                    MIN_TRIES = 205
                elif model == 'flex':
                    MIN_TRIES = 170
   

            # for model in ['fixed']:
                print(f'model = {model}-------------------------')
                if model == 'partflex':
                    for max_n_shifts in range(2,5):

                        if max_n_shifts == 2:
                            MIN_TRIES = 205
                        elif max_n_shifts == 3:
                            MIN_TRIES = 170
                        elif max_n_shifts == 4:
                            MIN_TRIES = 170
                    # for max_n_shifts in [4]:
                        objvalprev = 1e8
                        objval = 1e7
                        for trial_size in range(MIN_TRIES, MAX_TRIES):
                            workforce_dict = {0:trial_size, 1:1, 2:1, 3:1, 4:1}
                            if (objval < objvalprev)&(np.isnan(objval) == False):
                                objvalprev = objval
                                dict_results = run_roster_solver_objval(model, instance_file_weekday, shift_file_weekday, instance_file_weekend, shift_file_weekend, workforce_dict, OC, RM, GM, h_min, h_max, max_n_diff, max_n_shifts)
                                objval = dict_results['objective_value'][0]
                                print("trial_size",trial_size)
                                df_ = pd.DataFrame(dict_results)
                                df_['workforce_size_trial'] = trial_size
                                list_results.append(df_)
                            else:
                                #keep track of optimal
                                dict_out = dict_append_workforce(dict_out, base_file, city, demand_baseline, OC, RM, GM, model, max_n_shifts, trial_size-1, np.nan, objvalprev, objval)
                                break
                            if trial_size == int(MAX_TRIES-1):
                                #keep track of optimal
                                dict_out = dict_append_workforce(dict_out, base_file, city, demand_baseline, OC, RM, GM, model, max_n_shifts, trial_size, objvalprev, objval, np.nan)
                else:
                    objvalprev = 1e8
                    objval = 1e7
                    for trial_size in range(MIN_TRIES, MAX_TRIES):
                        workforce_dict = {0:trial_size, 1:1, 2:1, 3:1, 4:1}
                        if (objval < objvalprev)&(np.isnan(objval) == False):
                            objvalprev = objval
                            dict_results = run_roster_solver_objval(model, instance_file_weekday, shift_file_weekday, instance_file_weekend, shift_file_weekend, workforce_dict, OC, RM, GM, h_min, h_max, max_n_diff)
                            objval = dict_results['objective_value'][0]
                            print("trial_size",trial_size)
                            df_ = pd.DataFrame(dict_results)
                            df_['workforce_size_trial'] = trial_size
                            list_results.append(df_)
                        else: 
                            #keep track of optimal (trial size - 1)
                            dict_out = dict_append_workforce(dict_out, base_file, city, demand_baseline, OC, RM, GM, model, np.nan, trial_size-1, np.nan, objvalprev, objval)
                            break
                        if trial_size == int(MAX_TRIES-1):
                            #keep track of optimal
                            dict_out = dict_append_workforce(dict_out, base_file, city, demand_baseline, OC, RM, GM, model, np.nan, trial_size, objvalprev, objval, np.nan)
        #export optimal workforce size values
        with open(workforce_out, 'w') as f:
            json.dump(dict_out, f, indent=2)
        df_trials = pd.concat(list_results, ignore_index = True)
        df_trials.to_excel(f'../workforce_size/{base_file}_all_trials.xlsx', index = False)


paris_db=4.00============================================
OC = 1.2++++++++++++++++++++++
model = fixed-------------------------
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 65008 rows, 95168 columns and 2003868 nonzeros
Model fingerprint: 0x80179f10
Variable types: 33600 continuous, 61568 integer (61568 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [3e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+04]
Found heuristic solution: objective 2.240000e+12
Presolve removed 37053 rows and 32386 columns
Presolve time: 0.97s
Presolved: 27955 rows, 62782 columns, 312472 nonzeros
Found heuristic solution: objective 66181.397673
Variable types: 2512 continuous, 60270 integer (60270 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spi